<a href="https://colab.research.google.com/github/anjana-psvel/Precog_Task/blob/main/fine_tuning_gpt3_5_fin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [4]:
import pandas as pd
df = pd.read_csv("/content/train_data.csv")[:2000]
df.head(5)

,Unnamed: 0,phrase1,phrase2,sentence1,sentence2,label
0,0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0
1,1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1
2,2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0
3,3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1
4,4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0


In [121]:
import json

def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        # Create a dictionary with the label
        json_response = str(row['label'])


        # Append the user and assistant messages
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['sentence1'] + " and " + row['sentence2']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

# Assuming 'df' is your DataFrame
converted_data = convert_to_gpt35_format(df)
converted_data[0]['messages'][-1]['content']


str

In [105]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=df['label'],
    random_state=42  # for reproducibility
)



In [106]:
# # prompt: convert the df into jsonl file

def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
# write_to_jsonl(val_data, validation_file_name)


In [107]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [108]:
from openai import OpenAI
client = OpenAI(api_key="sk-proj-tmdJm64WtltHzFnvoSHxT3BlbkFJN4nho7DdkK5EVl00Ex7H")

In [109]:

training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-d36QVrR3oTFXACMTMFB6fE0i
Validation file id: file-FkszzUUwHDui1EETOq1U9m0S


In [112]:
suffix_name = "phrase_similarity"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response


FineTuningJob(id='ftjob-k2SMYLqZNjJeJBPrOOmz76YJ', created_at=1714796637, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-aNTe6eClTNdO7hYvuMfSaAm7', result_files=[], seed=1331893653, status='validating_files', trained_tokens=None, training_file='file-d36QVrR3oTFXACMTMFB6fE0i', validation_file='file-FkszzUUwHDui1EETOq1U9m0S', estimated_finish=None, integrations=[], user_provided_suffix='phrase_similarity')

In [115]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-k2SMYLqZNjJeJBPrOOmz76YJ', created_at=1714796637, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-aNTe6eClTNdO7hYvuMfSaAm7', result_files=[], seed=1331893653, status='running', trained_tokens=None, training_file='file-d36QVrR3oTFXACMTMFB6fE0i', validation_file='file-FkszzUUwHDui1EETOq1U9m0S', estimated_finish=1714799621, integrations=[], user_provided_suffix='phrase_similarity'), FineTuningJob(id='ftjob-i3mKBO6VCI4kLmPwP9ATBKts', created_at=1714796261, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-aNTe6eClTNdO7hYvuMfSaAm7'

In [124]:
response = client.fine_tuning.jobs.retrieve("ftjob-k2SMYLqZNjJeJBPrOOmz76YJ")
response

FineTuningJob(id='ftjob-k2SMYLqZNjJeJBPrOOmz76YJ', created_at=1714796637, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:phrase-similarity:9L1zQj8U', finished_at=1714799563, hyperparameters=Hyperparameters(n_epochs=3, batch_size=3, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-aNTe6eClTNdO7hYvuMfSaAm7', result_files=['file-jWzLJQD5aFISUGnyxqfJ2Ghl'], seed=1331893653, status='succeeded', trained_tokens=292143, training_file='file-d36QVrR3oTFXACMTMFB6fE0i', validation_file='file-FkszzUUwHDui1EETOq1U9m0S', estimated_finish=None, integrations=[], user_provided_suffix='phrase_similarity')

In [125]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal:phrase-similarity:9L1zQj8U


In [126]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['sentence1'] + " and " + row['sentence2']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content

In [127]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")

In [130]:
test_df = pd.read_csv("/content/test_set.csv")
store_predictions(test_df, fine_tuned_model_id)

fine_tuned_model_id ft:gpt-3.5-turbo-0125:personal:phrase-similarity:9L1zQj8U


In [129]:
# prompt: take 500 random samples from df and make it into a csv file named "test_set.csv"
from google.colab import files
test_df = df.sample(n=500, random_state=42)
test_df.to_csv('test_set.csv')
files.download('test_set.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [131]:
# prompt: predictions.csv as a df.

import pandas as pd

predictions_df = pd.read_csv("predictions.csv")

# Display the DataFrame
print(predictions_df.head())


   Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0            phrase1  \
0             0          1860        1860    value conflicts   
1             1           353         353   near elimination   
2             2          1333        1333    relative nature   
3             3           905         905     actual passage   
4             4          1289        1289  degraded position   

                       phrase2  \
0           moral disagreement   
1  almost complete eradication   
2         respective locations   
3                 real passing   
4             dissipated state   

                                           sentence1  \
0  For some critics, Islamic dress is an issue of...   
1  Buoyed by his near elimination, Bortolin made ...   
2  René Girard has critiqued the relative nature ...   
3  The actual passage through the rocks is now th...   
4  Yearsley deserves recognition as one who assis...   

                                           sentence2  label  Prediction  

In [132]:
#calculate accuracy score for label and prediction

from sklearn.metrics import accuracy_score

# Calculate accuracy score
accuracy = accuracy_score(predictions_df['label'], predictions_df['Prediction'])

# Print the accuracy score
print("Accuracy:", accuracy)


Accuracy: 0.914


In [133]:
predictions_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,phrase1,phrase2,sentence1,sentence2,label,Prediction
0,0,1860,1860,value conflicts,moral disagreement,"For some critics, Islamic dress is an issue of...","For some critics, Islamic dress is an issue of...",1,1
1,1,353,353,near elimination,almost complete eradication,"Buoyed by his near elimination, Bortolin made ...","Buoyed by his almost complete eradication, Bor...",0,0
2,2,1333,1333,relative nature,respective locations,René Girard has critiqued the relative nature ...,René Girard has critiqued the respective locat...,0,0
3,3,905,905,actual passage,real passing,The actual passage through the rocks is now th...,The real passing through the rocks is now the ...,0,0
4,4,1289,1289,degraded position,dissipated state,Yearsley deserves recognition as one who assis...,Yearsley deserves recognition as one who assis...,0,0
...,...,...,...,...,...,...,...,...,...
495,495,765,765,inferior position,unwanted role,The mouth of the fish is in the inferior posit...,The mouth of the fish is in the unwanted role.,0,0
496,496,1356,1356,similar device,same sort of implement,The fish hook or similar device has been made ...,The fish hook or same sort of implement has be...,1,1
497,497,408,408,public exchange,free forum,"In the simplest case, investability means that...","In the simplest case, investability means that...",1,1
498,498,1614,1614,fourth line,4th railway track,"The fourth line of the anthem reads ""Long live...","The 4th railway track of the anthem reads ""Lon...",0,0


In [134]:
# prompt: drop unnamed0 unamed1 and unamed2 columns

predictions_df.drop(['Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0'], axis=1, inplace=True)


In [135]:
predictions_df

,phrase1,phrase2,sentence1,sentence2,label,Prediction
0,value conflicts,moral disagreement,"For some critics, Islamic dress is an issue of...","For some critics, Islamic dress is an issue of...",1,1
1,near elimination,almost complete eradication,"Buoyed by his near elimination, Bortolin made ...","Buoyed by his almost complete eradication, Bor...",0,0
2,relative nature,respective locations,René Girard has critiqued the relative nature ...,René Girard has critiqued the respective locat...,0,0
3,actual passage,real passing,The actual passage through the rocks is now th...,The real passing through the rocks is now the ...,0,0
4,degraded position,dissipated state,Yearsley deserves recognition as one who assis...,Yearsley deserves recognition as one who assis...,0,0
...,...,...,...,...,...,...
495,inferior position,unwanted role,The mouth of the fish is in the inferior posit...,The mouth of the fish is in the unwanted role.,0,0
496,similar device,same sort of implement,The fish hook or similar device has been made ...,The fish hook or same sort of implement has be...,1,1
497,public exchange,free forum,"In the simplest case, investability means that...","In the simplest case, investability means that...",1,1
498,fourth line,4th railway track,"The fourth line of the anthem reads ""Long live...","The 4th railway track of the anthem reads ""Lon...",0,0
